# Data import for Peer_Massague_2020/Laughney dataset:

In [1]:
import os
os.chdir('./../')

In [2]:
import scanpy as sc
import anndata as ad
import pandas as pd
import preprocessing as pp

In [3]:
import h5py
import hdf5plugin
print(h5py.__version__)

2.10.0


In [4]:
adata_pd = pd.read_hdf('Laughney/laughney.h5', key='DF_ALL')

In [5]:
adata = sc.AnnData(X=adata_pd.values, obs=adata_pd.index.to_frame(), var=adata_pd.columns.to_frame())

In [6]:
adata.obs = adata.obs.set_index('Cell ID')

In [9]:
adata.obs.groupby('Sample ID').first()

,Legend,CELL_TYPE,Meta-Source,META_CELL_TYPE,PHENOGRAPH_CLASS
Sample ID,,,,,
0,RU653_TUMOR_1AS_UTR,Tm,TUMOR,Lymphoid,0_LYMPHOID
1,RU661_TUMOR_1AS_UTR,Th,TUMOR,Lymphoid,1_LYMPHOID
10,RU681_BRAINMET_4S_TR,DENDRITIC (ACTIVATED),MET,Myeloid,32_MYELOID_OTHER
11,RU682_TUMOR_2AS_UTR,Treg,TUMOR,Lymphoid,11_LYMPHOID
12,RU682_NORMAL_2AS_UTR,MACROPHAGE,NOR,Myeloid,0_MYELOID_OTHER
13,RU684_TUMOR_1AS_UTR,Tm,TUMOR,Lymphoid,2_LYMPHOID
14,RU684_NORMAL_1AS_UTR,Treg,NOR,Lymphoid,11_LYMPHOID
15,RU685_NORMAL_1ANS_UTR,NKT,NOR,Lymphoid,9_LYMPHOID
16,RU699_ADRENALMET_4,Treg,MET,Lymphoid,12_LYMPHOID


In [7]:
adata.obs.groupby('Meta-Source').size()

Meta-Source
MET       7655
NOR      11055
TUMOR    21795
dtype: int64

In [42]:
adata.obs['condition'] = adata.obs['Meta-Source']
adata.obs['sample'] = adata.obs['Sample ID']
adata.obs['subject_ID'] = adata.obs['Legend'].apply(lambda x: x.split('_')[0])
adata.obs['original_celltype_ann'] = adata.obs['CELL_TYPE']

del (
    adata.obs['Sample ID'], 
    adata.obs['Legend'], 
    adata.obs['CELL_TYPE'], 
    adata.obs['Meta-Source'],
    adata.obs['META_CELL_TYPE'],
    adata.obs['PHENOGRAPH_CLASS']
)

In [43]:
adata = adata[adata.obs['condition'] != 'MET']

In [44]:
del adata.var[0]

In [45]:
adata.obs['dataset'] = 'PeerMassague2020'
adata.obs['study'] = 'PeerMassague2020'

Trying to set attribute `.obs` of view, copying.


In [46]:
adata.obs.groupby('subject_ID').size()

subject_ID
RU653    2381
RU661    3669
RU675    5121
RU676    3439
RU679    1323
RU680     636
RU682    6927
RU684    6487
RU685    2867
dtype: int64

In [47]:
import scipy as sp

In [48]:
adata.X = sp.sparse.csr_matrix(adata.X)

In [49]:
adata.write('peermassague_full.h5ad')

... storing 'condition' as categorical
... storing 'sample' as categorical
... storing 'subject_ID' as categorical
... storing 'original_celltype_ann' as categorical
... storing 'dataset' as categorical
... storing 'study' as categorical


In [50]:
genes_subset = pd.read_csv('genes_for_mapping.csv')

In [51]:
adata_sub = pp.subset_and_pad_adata(genes_subset, adata)

not all genes were recovered, filling in 0 counts for 101 missing genes...


In [52]:
adata_sub.write('peermassague_subset.h5ad')

... storing 'gene_symbols' as categorical
